# Train Spider model

> (Estimated time: ~8 min with GPU)

In this vignette, we show how to train `spider` on spatial omics data to obtain independent omics embedding and spatial embedding of cells. We have following steps:

1. Init model
2. Register data
3. Fit model
4. Visualize results

 Here we use our mimics as an example, you can download it from [here](https://drive.google.com/file/d/1g2zUWNimhWHRZXjbIrZMi_6cbjKiHa0h/view?usp=sharing).


In [1]:
import scanpy as sc

from spider import Spider
from spider.utils import scanpy_viz

## Step 1: Init model
Init the model with a self-defined work_dir. It's better to have a directory that doesn't exist yet, `Spider` will create it and save the model, log and embedding files in it.

In [ ]:
model = Spider(work_dir='/path/to/work_dir')

## Step2: Register datasets
`Spider` expect `AnnData` object (or list of `AnnData`, see advanced tutorial) as input. The expression matrix should be raw counts (not log library size normalized or scaled). And the spati al coordinates should be in the `adata.obsm['spatial']` slot.

> See `AnnData` [documentation](https://anndata.readthedocs.io/en/latest/index.html) for more details if you are unfamiliar, including how to construct `AnnData` objects from scratch, and how to read data in other formats (csv, mtx, loom, etc.) into `AnnData` objects.

Here we load `AnnData` from existing `h5ad` file, which is the native file format for `AnnData`. The `h5ad` files used in this tutorial can be downloaded from here:

In [ ]:
adata = sc.read_h5ad('/path/to/adata')
model.register_data(adata)

## Step 3: fit model
Fit model with the following command:

In [ ]:
model.fit_omics()

After training, we can extract the embeddings from the model:

In [ ]:
gex_embedding = model.center_emb
nbr_embedding = model.nbr_emb

## Step 4: Visualize results

At last we cluster and visualize the embeddings.

In [ ]:
adata.obsm['X_center'] = gex_embedding
adata.obsm['X_nbr'] = nbr_embedding
adata = scanpy_viz(adata, ['center', 'nbr'], resolution=0.5)